# BBM469 : Data Intensive Applications Lab. Assignment 2

#### Student no : 21827497 - 21826464
#### Student name : Alperen Berk Işıldar - Uğurcan Erdoğan


## Table of Content

[Purpose](#purpose)   
[Data Understanding](#data_understanding)   
[Data Preparation](#data_preparation)   
[Modeling for Clustering](#clusteringmodel)  
[Clustering results](#clustering)    
[Modeling for Classification](#classificationmodel)  
[Evaluation](#evaluation)  
[References](#references)   


## Purpose <a class="anchor" id="purpose"></a>

Describe the aim of this assigment here. What are the questions you are trying to solve?

##### Packages

In [12]:
# Importing the necessary packages

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import warnings
import findspark
import pyspark
from pyspark.sql import SparkSession

warnings.filterwarnings("ignore")

findspark.init()

#Setting the objects spark and sc as stated in DataCamp
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Data Understanding<a class="anchor" id="data_understanding"></a>

In [29]:
fileName = "data.csv"
fileRDD = sc.textFile(fileName)
print("File path is: ", fileName)
print("fileRDD type is: ", type(fileRDD))
print("Total number of lines is: ", fileRDD.count())
print(fileRDD.take(5))

File path is:  data.csv
fileRDD type is:  <class 'pyspark.rdd.RDD'>
Total number of lines is:  13612
['Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class', '28395,610.291,208.1781167,173.888747,1.197191424,0.549812187,28715,190.1410973,0.763922518,0.988855999,0.958027126,0.913357755,0.007331506,0.003147289,0.834222388,0.998723889,SEKER', '28734,638.018,200.5247957,182.7344194,1.097356461,0.411785251,29172,191.2727505,0.783968133,0.984985603,0.887033637,0.953860842,0.006978659,0.003563624,0.909850506,0.998430331,SEKER', '29380,624.11,212.8261299,175.9311426,1.209712656,0.562727317,29690,193.4109041,0.778113248,0.989558774,0.947849473,0.908774239,0.007243912,0.003047733,0.825870617,0.999066137,SEKER', '30008,645.884,210.557999,182.5165157,1.153638059,0.498615976,30724,195.4670618,0.782681273,0.976695743,0.903936374,0.928328835,0.007016729,0.003214

In [32]:
df = spark.read.csv(fileName, header=True, inferSchema=True)
df.show(5)

+-----+---------+---------------+---------------+------------+------------+----------+-------------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+-----+
| Area|Perimeter|MajorAxisLength|MinorAxisLength|AspectRation|Eccentricity|ConvexArea|EquivDiameter|     Extent|   Solidity|  roundness|Compactness|ShapeFactor1|ShapeFactor2|ShapeFactor3|ShapeFactor4|Class|
+-----+---------+---------------+---------------+------------+------------+----------+-------------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+-----+
|28395|  610.291|    208.1781167|     173.888747| 1.197191424| 0.549812187|     28715|  190.1410973|0.763922518|0.988855999|0.958027126|0.913357755| 0.007331506| 0.003147289| 0.834222388| 0.998723889|SEKER|
|28734|  638.018|    200.5247957|    182.7344194| 1.097356461| 0.411785251|     29172|  191.2727505|0.783968133|0.984985603|0.887033637|0.953860842| 0.006978659| 0.00356362

In [46]:
df.printSchema()

root
 |-- Area: integer (nullable = true)
 |-- Perimeter: double (nullable = true)
 |-- MajorAxisLength: double (nullable = true)
 |-- MinorAxisLength: double (nullable = true)
 |-- AspectRation: double (nullable = true)
 |-- Eccentricity: double (nullable = true)
 |-- ConvexArea: integer (nullable = true)
 |-- EquivDiameter: double (nullable = true)
 |-- Extent: double (nullable = true)
 |-- Solidity: double (nullable = true)
 |-- roundness: double (nullable = true)
 |-- Compactness: double (nullable = true)
 |-- ShapeFactor1: double (nullable = true)
 |-- ShapeFactor2: double (nullable = true)
 |-- ShapeFactor3: double (nullable = true)
 |-- ShapeFactor4: double (nullable = true)
 |-- Class: string (nullable = true)



In [81]:
wordCountRDD = fileRDD.flatMap(lambda f: f.split(","))
wordCountRDD.collect()

['Area',
 'Perimeter',
 'MajorAxisLength',
 'MinorAxisLength',
 'AspectRation',
 'Eccentricity',
 'ConvexArea',
 'EquivDiameter',
 'Extent',
 'Solidity',
 'roundness',
 'Compactness',
 'ShapeFactor1',
 'ShapeFactor2',
 'ShapeFactor3',
 'ShapeFactor4',
 'Class',
 '28395',
 '610.291',
 '208.1781167',
 '173.888747',
 '1.197191424',
 '0.549812187',
 '28715',
 '190.1410973',
 '0.763922518',
 '0.988855999',
 '0.958027126',
 '0.913357755',
 '0.007331506',
 '0.003147289',
 '0.834222388',
 '0.998723889',
 'SEKER',
 '28734',
 '638.018',
 '200.5247957',
 '182.7344194',
 '1.097356461',
 '0.411785251',
 '29172',
 '191.2727505',
 '0.783968133',
 '0.984985603',
 '0.887033637',
 '0.953860842',
 '0.006978659',
 '0.003563624',
 '0.909850506',
 '0.998430331',
 'SEKER',
 '29380',
 '624.11',
 '212.8261299',
 '175.9311426',
 '1.209712656',
 '0.562727317',
 '29690',
 '193.4109041',
 '0.778113248',
 '0.989558774',
 '0.947849473',
 '0.908774239',
 '0.007243912',
 '0.003047733',
 '0.825870617',
 '0.999066137',


In [82]:
wordCountRDD = wordCountRDD.map(lambda f: (f,1))
wordCountRDD.collect()

[('Area', 1),
 ('Perimeter', 1),
 ('MajorAxisLength', 1),
 ('MinorAxisLength', 1),
 ('AspectRation', 1),
 ('Eccentricity', 1),
 ('ConvexArea', 1),
 ('EquivDiameter', 1),
 ('Extent', 1),
 ('Solidity', 1),
 ('roundness', 1),
 ('Compactness', 1),
 ('ShapeFactor1', 1),
 ('ShapeFactor2', 1),
 ('ShapeFactor3', 1),
 ('ShapeFactor4', 1),
 ('Class', 1),
 ('28395', 1),
 ('610.291', 1),
 ('208.1781167', 1),
 ('173.888747', 1),
 ('1.197191424', 1),
 ('0.549812187', 1),
 ('28715', 1),
 ('190.1410973', 1),
 ('0.763922518', 1),
 ('0.988855999', 1),
 ('0.958027126', 1),
 ('0.913357755', 1),
 ('0.007331506', 1),
 ('0.003147289', 1),
 ('0.834222388', 1),
 ('0.998723889', 1),
 ('SEKER', 1),
 ('28734', 1),
 ('638.018', 1),
 ('200.5247957', 1),
 ('182.7344194', 1),
 ('1.097356461', 1),
 ('0.411785251', 1),
 ('29172', 1),
 ('191.2727505', 1),
 ('0.783968133', 1),
 ('0.984985603', 1),
 ('0.887033637', 1),
 ('0.953860842', 1),
 ('0.006978659', 1),
 ('0.003563624', 1),
 ('0.909850506', 1),
 ('0.998430331', 1),

In [83]:
wordCountRDD = wordCountRDD.reduceByKey(lambda f,h: f+h)
wordCountRDD = wordCountRDD.sortBy(lambda f: f[1], ascending=False)
wordCountRDD.collect()

[('DERMASON', 3546),
 ('SIRA', 2636),
 ('SEKER', 2027),
 ('HOROZ', 1928),
 ('CALI', 1630),
 ('BARBUNYA', 1322),
 ('BOMBAY', 522),
 ('40504', 6),
 ('47446', 6),
 ('41706', 6),
 ('39215', 6),
 ('29172', 5),
 ('34774', 5),
 ('35845', 5),
 ('36461', 5),
 ('37023', 5),
 ('38554', 5),
 ('38760', 5),
 ('44704', 5),
 ('45454', 5),
 ('33518', 5),
 ('44199', 5),
 ('28122', 5),
 ('31423', 5),
 ('32951', 5),
 ('34594', 5),
 ('35354', 5),
 ('36609', 5),
 ('38542', 5),
 ('38426', 5),
 ('39175', 5),
 ('44515', 5),
 ('47134', 5),
 ('47605', 5),
 ('27997', 5),
 ('30529', 5),
 ('33338', 4),
 ('33788', 4),
 ('34246', 4),
 ('34509', 4),
 ('34827', 4),
 ('34865', 4),
 ('35298', 4),
 ('35252', 4),
 ('35414', 4),
 ('212.4291787', 4),
 ('214.4187649', 4),
 ('36553', 4),
 ('36805', 4),
 ('36883', 4),
 ('37052', 4),
 ('37240', 4),
 ('37393', 4),
 ('38222', 4),
 ('38307', 4),
 ('37968', 4),
 ('38617', 4),
 ('221.1911001', 4),
 ('38941', 4),
 ('39295', 4),
 ('38945', 4),
 ('39528', 4),
 ('39285', 4),
 ('39861', 4

Provide analysis and statistical information about the data set. How much data is there? What are the data types? What are the largest, smallest and average values of these data?

## Data Preparation<a class="anchor" id="data_preparation"></a>

Explain what kind of data transformations, feature selection and/or engineering you will perform.

## Modeling for Clustering <a class="anchor" id="clusteringmodel"></a>

Which model will be used? Why? What parameters?

## Clustering results<a class="anchor" id="clustering"></a>

Present the clustering results and graphs.

Evaluate your Clustering model. Provide results, tables, charts, etc.


## Modeling for Classification <a class="anchor" id="classificationmodel"></a>

Which model will be used? Why? What parameters?

## Evaluation<a class="anchor" id="evaluation"></a>

Evaluate your model. Provide results, tables, charts, etc.

## References<a class="anchor" id="references"></a>

List all the sources you used during your work.
This includes jupyter notebooks you found on Internet.
Remeber, your work may not be an original one. However, this document must be genuine. Copy and paste kind of deliveries will be punished badly.


**Disclaimer!** <font color='grey'>This notebook was prepared by <student name(s)> as an assigment for the *BBM469 - Data Intensive Applications Laboratory* class. The notebook is available for educational purposes only. There is no guarantee on the correctness of the content provided as it is a student work.

If you think there is any copyright violation, please let us [know](https://forms.gle/BNNRB2kR8ZHVEREq8). 
</font>